[인공지능] - 윤수식

Homework2

1. KeyBERT를 이용한 Keyword 추출 및 분석

In [ ]:
# 기본적인 모델링 방식은 실습 방식을 기준으로 하였습니다.

!pip install keybert

from keybert import KeyBERT

doc = """In a rare display of camaraderie among global business titans, Nvidia CEO Jensen Huang,
Samsung Electronics Chairman Lee Jae-yong and Hyundai Motor Group Executive Chair Chung Euisun
gathered for a casual dinner of Korean fried chicken and beer in Seoul on Thursday evening --
a meeting that turned heads and drew crowds.

The three moguls met at around 7:30 p.m. at Kkanbu Chicken, a popular restaurant near
Samseong Station on Subway Line No. 2, in the affluent Gangnam district. “Kkanbu,”
a slang term for “close friend,” rose to global fame after its mention in the Netflix hit series Squid Game.
According to industry sources, the venue was selected by Nvidia."""

# KeyBERT 모델을 초기화
kw_model = KeyBERT(model='all-mpnet-base-v2')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# --- [요구사항 (a): 전체 문서에서 키워드 10개 추출] ---
print("--- [요구사항 (a): 전체 문서 키워드 Top 10] ---")
keywords_full = kw_model.extract_keywords(
    doc,
    keyphrase_ngram_range=(1, 1),  # 단어 수 범위
    stop_words='english',          # 불용어 처리
    top_n=10                       # n개 단어 추출
)

import pandas as pd

# 추출된 키워드 리스트를 DataFrame으로 변환 및 출력
df_full = pd.DataFrame(keywords_full, columns=['Keyword', 'Similarity (Score)'])
print(df_full.to_string(index=False))

--- [요구사항 (a): 전체 문서 키워드 Top 10] ---
   Keyword  Similarity (Score)
    kkanbu              0.5602
restaurant              0.3993
    dinner              0.3882
   meeting              0.3805
       ceo              0.3712
      yong              0.3264
    korean              0.3221
     huang              0.3210
   chicken              0.3208
   gangnam              0.3032


In [ ]:
# --- [요구사항 (b): 문서를 단어 수 기준으로 2등분하여 각 5개씩 추출] ---
print("\n--- [요구사항 (b): 단어 수 기준 2등분 추출] ---")

# 단어 리스트의 정확한 중간 지점을 계산
words = doc.split()
midpoint = len(words) // 2

# 단어 리스트를 기준으로 두 개의 텍스트 부분(doc_part1, doc_part2)을 만듦
doc_part1 = " ".join(words[:midpoint])
doc_part2 = " ".join(words[midpoint:])

# 첫 번째 부분에서 키워드 5개를 추출
print("\n--- [문서 1/2 키워드 Top 5] ---")
keywords_part1 = kw_model.extract_keywords(
    doc_part1,
    keyphrase_ngram_range=(1, 1),
    stop_words='english',
    top_n=5
)
df_part1 = pd.DataFrame(keywords_part1, columns=['Keyword', 'Similarity (Score)'])
print(df_part1.to_string(index=False))

# 두 번째 부분에서 키워드 5개를 추출
print("\n--- [문서 2/2 키워드 Top 5] ---")
keywords_part2 = kw_model.extract_keywords(
    doc_part2,
    keyphrase_ngram_range=(1, 1),
    stop_words='english',
    top_n=5
)
df_part2 = pd.DataFrame(keywords_part2, columns=['Keyword', 'Similarity (Score)'])
print(df_part2.to_string(index=False))


--- [요구사항 (b): 단어 수 기준 2등분 추출] ---

--- [문서 1/2 키워드 Top 5] ---
  Keyword  Similarity (Score)
      ceo              0.5014
  meeting              0.4369
    huang              0.4021
executive              0.3683
   moguls              0.3673

--- [문서 2/2 키워드 Top 5] ---
   Keyword  Similarity (Score)
    kkanbu              0.6423
   gangnam              0.3971
restaurant              0.3764
     squid              0.3674
    subway              0.3318


In [ ]:
import pandas as pd
import numpy as np

# ------------------------------
print("--- [분석 1: 전체 vs 분할 키워드 비교] ---")

# (b)의 분할된 키워드 결과(part1, part2)를 하나로 병합
df_segmented = pd.concat([df_part1, df_part2]).reset_index(drop=True)

# (a)와 (b)의 결과를 'Keyword'를 기준으로 병합
df_comparison = pd.merge(
    df_full.rename(columns={'Similarity (Score)': 'Score_FullDoc (a)'}),
    df_segmented.rename(columns={'Similarity (Score)': 'Score_Segmented (b)'}),
    on='Keyword',
    how='outer'
)

# 점수를 기준으로 내림차순 정렬
df_comparison = df_comparison.sort_values(by=['Score_FullDoc (a)', 'Score_Segmented (b)'], ascending=False)

print("Score_FullDoc (a): (a) 전체 문서에서 추출된 점수")
print("Score_Segmented (b): (b) 분할 문서에서 추출된 점수")
print(df_comparison.to_string(index=False))


# ------------------------------
print("\n\n--- [분석 2: 고유 키워드 확인] ---")

keywords_set_full = set(df_full['Keyword'])
keywords_set_segmented = set(df_segmented['Keyword'])

# (a)에만 있는 키워드
unique_to_full = keywords_set_full - keywords_set_segmented
print("\n(a) 전체 문서에만 있는 키워드:")
if unique_to_full:
    print(list(unique_to_full))
else:
    print("없음")

# (b)에만 있는 키워드
unique_to_segmented = keywords_set_segmented - keywords_set_full
print("\n(b) 분할 문서에만 있는 키워드:")
if unique_to_segmented:
    print(list(unique_to_segmented))
else:
    print("없음")


# ------------------------------
print("\n\n--- [분석 3: 공통 키워드 스코어 비교] ---")

# 공통 키워드 목록
common_keywords = keywords_set_full.intersection(keywords_set_segmented)

if common_keywords:
    df_common = df_comparison.dropna()
    print(df_common.to_string(index=False))
else:
    print("공통으로 추출된 키워드가 없습니다.")

--- [분석 1: 전체 vs 분할 키워드 비교] ---
Score_FullDoc (a): (a) 전체 문서에서 추출된 점수
Score_Segmented (b): (b) 분할 문서에서 추출된 점수
   Keyword  Score_FullDoc (a)  Score_Segmented (b)
    kkanbu             0.5602               0.6423
restaurant             0.3993               0.3764
    dinner             0.3882                  NaN
   meeting             0.3805               0.4369
       ceo             0.3712               0.5014
      yong             0.3264                  NaN
    korean             0.3221                  NaN
     huang             0.3210               0.4021
   chicken             0.3208                  NaN
   gangnam             0.3032               0.3971
 executive                NaN               0.3683
     squid                NaN               0.3674
    moguls                NaN               0.3673
    subway                NaN               0.3318


--- [분석 2: 고유 키워드 확인] ---

(a) 전체 문서에만 있는 키워드:
['chicken', 'yong', 'korean', 'dinner']

(b) 분할 문서에만 있는 키워드:
['subway', 'mog